In [64]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import numpy as np
import math
from nltk.corpus import stopwords

In [53]:
# set path 
os.chdir(r"C:\Users\Public")
path = os.getcwd()
print(path)

C:\Users\Public


In [79]:
# load train data frame with ID index
stopwords = stopwords.words('english')
df_train_original = pd.read_csv (r"ffp_train.csv",index_col=0)
df_train_original.head()

,OTHER_SITE_VALUE,STATUS_PANTINUM,STATUS_GOLD,STATUS_SILVER,NUM_DEAL,LAST_DEAL,ADVANCE_PURCHASE,FARE_L_Y1,FARE_L_Y2,FARE_L_Y3,...,POINTS_L_Y1,POINTS_L_Y2,POINTS_L_Y3,POINTS_L_Y4,POINTS_L_Y5,CALL_FLAG,CREDIT_PROBLEM,RETURN_FLAG,BENEFIT_FLAG,BUYER_FLAG
ID,,,,,,,,,,,,,,,,,,,,,
1,13.196559,0,0,0,4,43.3,26,140.0,149.2,161.7,...,77.8,94.8,114.2,167.1,154.1,0,0,0,0,0
2,10.284568,0,0,1,2,123.3,22,241.7,241.7,235.0,...,220.2,242.8,244.4,225.5,223.6,0,1,0,1,0
3,11.276389,0,0,0,5,30.0,20,144.2,150.0,150.0,...,158.7,159.8,175.1,152.3,180.1,0,0,0,0,0
4,10.678160,0,0,0,4,40.0,21,145.0,148.3,150.0,...,109.6,127.7,135.1,110.4,139.5,0,0,0,0,0
5,9.170843,0,0,0,5,30.0,22,133.3,135.0,132.5,...,94.5,84.4,94.9,85.2,84.8,0,0,0,0,0


In [86]:
# load reviews data frame with ID index
df_review_original = pd.read_csv (r"reviews_training.csv",index_col=0)
df_train_HW = pd.read_csv (r"text_training.csv",index_col=0)
df_train_HW.head()

,somehow,ketchup,afternoon,part,cover,shame,sunflow,excess,puck,letharg,...,unabl,arriv,allbr,understand,histori,workout,hull,french,planet,rating
ID,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [87]:
# calculate idf for each word in bage of words and place it in a dictionary
docs = len(df_train_HW)
dict_idf = {}
for column in df_train_HW:
    count_doc = 0 
    for i in df_train_HW[column]:
        if i > 0:
            count_doc +=1
    if count_doc != 0:
        idf = math.log(docs/count_doc)
    else:
        idf = 0 
    dict_idf[column] = idf
    
print(dict_idf)


{'somehow': 5.403677882205863, 'ketchup': 6.502290170873972, 'afternoon': 5.403677882205863, 'part': 3.6888794541139363, 'cover': 4.8283137373023015, 'shame': 4.961845129926823, 'sunflow': 5.809142990314028, 'excess': 6.214608098422191, 'puck': 6.907755278982137, 'letharg': 6.907755278982137, 'dont': 1.867561182644337, 'breast': 6.214608098422191, 'deliv': 4.104394898075602, 'mad': 5.809142990314028, 'bag': 2.277892480403674, 'holiday': 5.2030071867437115, 'night': 4.305065593537753, 'constant': 5.298317366548036, 'back': 2.617295837833746, 'spit': 5.035953102080546, 'tri': 1.3528595850336533, 'dirti': 6.502290170873972, 'stronger': 5.115995809754082, 'kosher': 5.654992310486769, 'most': 2.501036031717884, 'pocket': 5.521460917862246, 'can': 1.5582696258597009, 'finish': 4.382026634673881, 'believ': 3.7090821614314557, 'equal': 5.2030071867437115, 'oatmeal': 4.605170185988092, 'bubbl': 5.991464547107982, 'vacuum': 5.991464547107982, 'write': 4.509860006183766, 'rancid': 4.6051701859880

In [40]:
# calculating tf-idf for each word for df_review_original 
j = 0
list_tf = []
DF_main = pd.DataFrame() 
column_list= list(df_review_original.iloc[:,1:].columns)
DF_trial_1= pd.DataFrame(columns= list(df_review_original.columns))
for i in df_review_original.iterrows():
    total_words = df_review_original.iloc[j,:].sum()
    j +=1
    for column in df_review_original.iloc[:,1:]:
        a = df_review_original.iloc[j-1][column]
        tf_score = (a/total_words)
        idf_score = dict_idf[column]
        tf_idf_score = idf_score*tf_score
        list_tf.append(tf_idf_score)
    DF_trial =  pd.DataFrame(list_tf, index= column_list).T
    DF_main = DF_main.append(DF_trial)
    list_tf = []

DF_main = DF_main.reset_index(drop=True)
DF_main.head(15)

,ketchup,afternoon,part,cover,shame,sunflow,excess,puck,letharg,dont,...,that,unabl,arriv,allbr,understand,histori,workout,hull,french,planet
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.013851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.017659,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10735,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
5,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.019091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
6,0.0,0.0,0.0,0.067622,0.0,0.0,0.0,0.0,0.0,0.013889,...,0.011124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
7,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.011393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
8,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.014128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
9,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0


In [59]:
# add client id to the index for df_train_original
df_train_reviews_tf_idf_id_index = pd.read_csv (r"df_review_tf_if_number_index.csv",index_col=0)
list_id = list(df_review_original.index.values)
df_train_reviews_tf_idf_id_index['ID'] = list_id
df_train_reviews_tf_idf_id_index.set_index('ID', drop=True, inplace=True)
df_train_reviews_tf_idf_id_index[200:].head(10)

,ketchup,afternoon,part,cover,shame,sunflow,excess,puck,letharg,dont,...,that,unabl,arriv,allbr,understand,histori,workout,hull,french,planet
ID,,,,,,,,,,,,,,,,,,,,,
4678,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.006792,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4688,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.013851,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4696,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4705,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.08808,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4708,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.017659,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4713,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4727,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4736,0.0,0.0,0.039713,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4772,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.035319,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
# calculating tf-idf for each word for df_train_HW
j = 0
list_tf = []
DF_main_HW = pd.DataFrame() 
column_list= list(df_train_HW.iloc[:,1:2500].columns)
for i in df_train_HW.iterrows():
    total_words = df_train_HW.iloc[j,:].sum()
    j +=1
    for column in df_train_HW.iloc[:,1:2500]:
        a = df_train_HW.iloc[j-1][column]
        tf_score = (a/total_words)
        idf_score = dict_idf[column]
        tf_idf_score = idf_score*tf_score
        list_tf.append(tf_idf_score)
    DF_trial =  pd.DataFrame(list_tf, index= column_list).T
    DF_main_HW = DF_main_HW.append(DF_trial)
    list_tf = []

DF_main_HW.reset_index(drop=True, inplace=True)
DF_main_HW.head(15)

,ketchup,afternoon,part,cover,shame,sunflow,excess,puck,letharg,dont,...,that,unabl,arriv,allbr,understand,histori,workout,hull,french,planet
0,0.0,0.0,0.0,0.109734,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.014256,...,0.028858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.035915,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.023146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.019387,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.036981,...,0.007486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.079552,0.0
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.010081,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [72]:
# bag of words version stopwords removed for df_train_original
column_list= list(df_review_original.iloc[:,1:].columns)
delete = []
for word in stopwords:
    if word in column_list:
        delete.append(word)
df_train_review_original_no_stop_words = df_review_original.drop(delete, axis = 1)        
#print(delete)
df_train_review_original_no_stop_words.head()

,somehow,ketchup,afternoon,part,cover,shame,sunflow,excess,puck,letharg,...,toler,unabl,arriv,allbr,understand,histori,workout,hull,french,planet
ID,,,,,,,,,,,,,,,,,,,,,
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
56,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
# tf-idf version stopwords removed for df_train_original
df_train_reviews_tf_idf_id_index_no_stop_words = pd.read_csv (r"df_train_reviews_tf_idf_id_index.csv",index_col=0)
column_list= list(df_review_original.iloc[:,1:].columns)
delete = []
for word in stopwords:
    if word in column_list:
        delete.append(word)
df_train_reviews_tf_idf_id_index_no_stop_words = df_train_reviews_tf_idf_id_index.drop(delete, axis = 1)        
#print(delete)
df_train_reviews_tf_idf_id_index_no_stop_words.head()

,ketchup,afternoon,part,cover,shame,sunflow,excess,puck,letharg,dont,...,toler,unabl,arriv,allbr,understand,histori,workout,hull,french,planet
ID,,,,,,,,,,,,,,,,,,,,,
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10735,0.0
56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0


In [114]:
# tf-idf version stopwords removed for DF_main_HW
column_list_df_train_HW= list(DF_main_HW.iloc[:,1:2500].columns)
delete = []
for word in stopwords:
    if word in column_list_df_train_HW:
        delete.append(word)
df_train_HW_tf_idf_without_stopwords = DF_main_HW.drop(delete, axis = 1)        
print(delete)
df_train_HW_tf_idf_without_stopwords.head()

['myself', 'our', 'you', 'your', 'yourself', 'him', 'his', 'she', 'her', 'herself', 'itself', 'they', 'them', 'their', 'what', 'which', 'who', 'this', 'that', 'these', 'those', 'are', 'was', 'were', 'been', 'have', 'has', 'had', 'did', 'the', 'and', 'but', 'until', 'while', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'after', 'below', 'from', 'down', 'out', 'off', 'over', 'under', 'again', 'further', 'then', 'here', 'there', 'when', 'where', 'how', 'all', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'nor', 'not', 'own', 'same', 'than', 'too', 'can', 'will', 'just', 'should', 'now']


,ketchup,afternoon,part,cover,shame,sunflow,excess,puck,letharg,dont,...,toler,unabl,arriv,allbr,understand,histori,workout,hull,french,planet
0,0.0,0.0,0.0,0.109734,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.014256,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.035915,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
# count version stopwords removed for df_train_HW
column_list_df_train_HW= list(df_train_HW.iloc[:,1:2501].columns)
delete = []
for word in stopwords:
    if word in column_list_df_train_HW:
        delete.append(word)
df_train_HW_count_no_stop_words = df_train_HW.drop(delete, axis = 1)        
print(delete)
df_train_HW_count_no_stop_words.head()

['myself', 'our', 'you', 'your', 'yourself', 'him', 'his', 'she', 'her', 'herself', 'itself', 'they', 'them', 'their', 'what', 'which', 'who', 'this', 'that', 'these', 'those', 'are', 'was', 'were', 'been', 'have', 'has', 'had', 'did', 'the', 'and', 'but', 'until', 'while', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'after', 'below', 'from', 'down', 'out', 'off', 'over', 'under', 'again', 'further', 'then', 'here', 'there', 'when', 'where', 'how', 'all', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'nor', 'not', 'own', 'same', 'than', 'too', 'can', 'will', 'just', 'should', 'now']


,somehow,ketchup,afternoon,part,cover,shame,sunflow,excess,puck,letharg,...,unabl,arriv,allbr,understand,histori,workout,hull,french,planet,rating
ID,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [115]:
# save the file
df_train_HW_tf_idf_without_stopwords.to_csv(r'C:\Users\Public\df_train_HW_reviews_tf_idf_id_index_without_stopwords.csv', index = True)